In [1]:
import tensorflow as tf
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json

In [2]:
#讀檔

num = 0
train_data = []
test_data = []
train_size = 4000
test_size = 100
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

input_file = open('translation2019zh_train.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    if num > train_size + test_size:
        break
    elif num > train_size:
        test_data.append(json.loads(line))
    else:
        train_data.append(json.loads(line))
input_file.close()

'''
input_file = open('translation2019zh_valid.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    #data.append(json.loads(line))
input_file.close()'''

"\ninput_file = open('translation2019zh_valid.json', encoding = 'utf8')\nfor line in input_file.readlines():\n    num = num + 1\n    #data.append(json.loads(line))\ninput_file.close()"

In [3]:
#簡轉繁
from opencc import OpenCC

cc = OpenCC('s2t')

for data in train_data:
    data['chinese'] = cc.convert(data['chinese'])

In [4]:
# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for item in train_data:
    input_text = item['english']
    target_text = '\t' + item['chinese'] + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
# 字典排序            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [6]:
# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 4000
Number of unique input tokens: 259
Number of unique output tokens: 3461
Max sequence length for inputs: 256
Max sequence length for outputs: 144


In [8]:
# 以dict儲存字典單字及序號
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [9]:
# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [10]:
# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [11]:
# 建立 encoder LSTM 隱藏層
origin_encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_0')
origin_encoder = LSTM(latent_dim, return_state=True)
origin_encoder_outputs, origin_state_h, origin_state_c = origin_encoder(origin_encoder_inputs)
origin_encoder_states = [origin_state_h, origin_state_c]

encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_1')
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
print(encoder_outputs)
print(state_h)
print(state_c)
encoder_outputs.type_spec

KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:0', description="created by layer 'lstm_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:2', description="created by layer 'lstm_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:3', description="created by layer 'lstm_1'")


TensorSpec(shape=(None, 256), dtype=tf.float32, name=None)

In [12]:
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

In [13]:
# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='input_2')
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
# 訓練
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
50/50 [==============================] - 432s 7s/step - loss: 1.8255 - val_loss: 1.7299
Epoch 2/100
50/50 [==============================] - 368s 7s/step - loss: 1.6830 - val_loss: 1.7226
Epoch 3/100
50/50 [==============================] - 355s 7s/step - loss: 1.7120 - val_loss: 1.7199
Epoch 4/100
50/50 [==============================] - 367s 7s/step - loss: 1.7110 - val_loss: 1.7187
Epoch 5/100
50/50 [==============================] - 363s 7s/step - loss: 1.7049 - val_loss: 1.7174
Epoch 6/100
50/50 [==============================] - 375s 8s/step - loss: 1.7011 - val_loss: 1.7188
Epoch 7/100
50/50 [==============================] - 376s 8s/step - loss: 1.6890 - val_loss: 1.7179
Epoch 8/100
50/50 [==============================] - 367s 7s/step - loss: 1.6998 - val_loss: 1.7166
Epoch 9/100
50/50 [==============================] - 369s 7s/step - loss: 1.6819 - val_loss: 1.7163
Epoch 10/100
50/50 [==============================] - 374s 8s/step - loss: 1.7077 - val_loss: 1.7174

50/50 [==============================] - 411s 8s/step - loss: 1.6702 - val_loss: 1.7000
Epoch 82/100
50/50 [==============================] - 490s 10s/step - loss: 1.6880 - val_loss: 1.7026
Epoch 83/100
50/50 [==============================] - 411s 8s/step - loss: 1.6488 - val_loss: 1.7001
Epoch 84/100
50/50 [==============================] - 491s 10s/step - loss: 1.6677 - val_loss: 1.7082
Epoch 85/100
50/50 [==============================] - 428s 8s/step - loss: 1.6825 - val_loss: 1.7097
Epoch 86/100
50/50 [==============================] - 422s 8s/step - loss: 1.6851 - val_loss: 1.7066
Epoch 87/100
50/50 [==============================] - 422s 8s/step - loss: 1.6763 - val_loss: 1.7016
Epoch 88/100
50/50 [==============================] - 386s 8s/step - loss: 1.6630 - val_loss: 1.7004
Epoch 89/100
50/50 [==============================] - 495s 10s/step - loss: 1.6622 - val_loss: 1.6977
Epoch 90/100
50/50 [==============================] - 418s 8s/step - loss: 1.6645 - val_loss: 1.7009


In [16]:
# 定義編碼器取樣模型
encoder_model = Model(encoder_inputs, encoder_states)

In [17]:
# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [18]:
# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

In [19]:
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

print([decoder_inputs] + decoder_states_inputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
'''
decoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))
'''

[<KerasTensor: shape=(None, None, 3461) dtype=float32 (created by layer 'input_2')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_3')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_4')>]


'\ndecoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))\n'

In [20]:
# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [21]:
# 模型預測，並取得翻譯結果(中文)    
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [22]:
test_texts = []
test_characters = set()
for text in test_data:
    test_texts.append(text['english'])
    for char in text['english']:
        if char not in test_characters:
            test_characters.add(char)

In [23]:
test_characters = sorted(list(test_characters))
num_test = len(test_characters)

In [24]:
test_index = dict([(char, i) for i, char in enumerate(test_characters)])

In [25]:
test_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

In [26]:
# 設定 encoder_input、decoder_input對應的順序    
for i, test_text in enumerate(test_texts):
    for t, char in enumerate(test_text):
        #print(char)
        #print(input_token_index[char])
        test_input_data[i, t, input_token_index[char]] = 1.

In [28]:
# 測試100次
for seq_index in range(len(test_texts)):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(seq_index + 1, '.')
    print('English:', input_texts[seq_index])
    try:
        print('Chinese:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Chinese:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])


1 .
English: For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
2 .
English: He calls the Green Book, his book of teachings, “the new gospel.
Chinese: 我們在在在的的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
3 .
English: And the light breeze moves me to caress her long ear
Chinese: 我們在在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
4 .
English: They have the blood of martyrs is the White to flow …
Chinese: 我們在在的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
5 .
English: Finally, the Lakers head to the 

32 .
English: Yesterday, a city with the husband and wife suffering from AIDS in the city hospital to get two of their 18-month-old daughter of the AIDS antibody test results of the report alone.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
33 .
English: A model wearing a traditional Korean hanbok performs in a water tank at the "Underwater Hanbok Fashion Show" in Seoul, South Korea.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
34 .
English: Haven't found some hurt you when it is not pain, you pay attention to it begins to faint do painful.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
35 .
English: First of all, the term "justice" can have different interpretati

60 .
English: He became in legitimately through the door of the Law (vv. 1-3).
Chinese: 我們在在在的的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
61 .
English: The effect mechanism of laser biology was systemotically and deeply discussed in this artical, from 4 aspects:the light, electromagnetism field, heat and pressure effect of laser.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
62 .
English: The variant of FOXO3A associated with longevity is much more prevalent in 100-year-olds even than in 95-year-olds, which clearly demonstrates the value of studying the centenarian genome.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
63 .
English: Again, Uruguay are slight exception – they did

88 .
English: We've rounded up some unusual ways to put your bottom-shelf vodka to good use all around your house.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
89 .
English: The target of anti_monopoly law should be to contain monopolizing behavior and various restrictive practices.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
90 .
English: To build a conservation-minded society, we should act at present for our future.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，
91 .
English: Large doses of carbs, sugar, and caffeine might keep you awake for a short time, but they will eventually lead to a "crash, " and have the opposite effect.
Chinese: 我們在在在的的的，，，，，，，，，，，，，，